# Accidentes de tráfico en Reino Unido entre 2010 y 2014 

### Disponible en Kaggle en:
https://www.kaggle.com/stefanoleone992/adm-project-road-accidents-in-uk

### Variables y significado

* Accident_Index: Accident index
* Latitude: Accident latitude
* Longitude: Accident longitude
* Region: Accident region
* Urban_or_Rural_Area: Accident area (rural or urban)
* X1st_Road_Class: Accident road class
* Driver_IMD_Decile: Road IMD Decile
* Speed_limit: Road speed limit
* Road_Type: Road type
* Road_Surface_Conditions: Road surface condition
* Weather: Weather
* High_Wind: High wind
* Lights: Road lights
* Datetime: Accident datetime
* Year: Accident year
* Season: Accident season
* Month_of_Year: Accident month
* Day_of_Month: Accident day of month
* Day_of_Week: Accident day of week
* Hour_of_Day: Accident hour of day
* Number_of_Vehicles: Accident number of vehicles
* Age_of_Driver: Driver age
* Age_of_Vehicle: Vehicle age
* Junction_Detail: Accident junction detail
* Junction_Location: Accident junction location
* X1st_Point_of_Impact: Vehicle first point of impact
* Driver_Journey_Purpose: Driver journey purpose
* Engine_CC: Vehicle engine power (in CC)
* Propulsion_Code: Vehicle propulsion code
* Vehicle_Make: Vehicle brand
* Vehicle_Category: Vehicle brand category
* Vehicle_Manoeuvre: Vehicle manoeuvre when accident happened
* Accident_Severity: Accident severity

**Nombre completo del alumno:**  Jonás De Martín Rodríguez.

# INSTRUCCIONES 

En cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). 

**No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

**Nunca se debe redondear ninguna cantidad si no lo pide explícitamente el enunciado**

### Cada solución debe escribirse obligatoriamente en la celda habilitada para ello. Cualquier celda adicional que se haya creado durante el desarrollo deberá ser eliminada.

Si necesitas crear celdas auxiliares durante el desarrollo, puedes hacerlo pero debes asegurarte de borrarlas antes de entregar el notebook.

### Sobre el dataset anterior (accidents_uk.csv) se pide:

**Ejercicio 1 (1.5 puntos)** 
* Leerlo tratando de que Spark infiera el tipo de dato de cada columna.
* Crear una columna llamada `Age_Category` renombrando los valores de la columna `Age_of_Driver` donde los valores 1 y 2 de la columna original sean etiquetados en la columna nueva como "Adolescente", los valores 3 y 4 como "Joven", los valores 5 y 6 como "Adulto", y los valores 7 y 8 como "Anciano".
* Crear una columna llamada `hora` aplicando la función `F.hour` a la columna `"Datetime"` ya existente.
* El resultado debe guardarse **cacheado** en la variable `accidentesDF`.

In [0]:
# Importo el módulo de funciones de PySpark SQL, que contiene transformaciones como when, col, hour, count, etc.
# Uso el alias 'F' para referirme a esas funciones de forma abreviada (por ejemplo: F.col("columna")).
from pyspark.sql import functions as F

# Configuración para acceder a Azure Data Lake Storage (ADLS Gen2)
spark.conf.set(
    "fs.azure.account.key.masterjmr.dfs.core.windows.net",
    "<REDACTED_ACCESS_KEY>"
)

# Ruta del archivo CSV en ADLS
ruta_csv = "abfss://datos@masterjmr.dfs.core.windows.net/accidents_uk.csv"

# Lectura del CSV desde ADLS, con inferencia de tipos
accidentesDF = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(ruta_csv)

    # Creo una nueva columna llamada 'Age_Category' clasificando la edad del conductor en 4 grupos.
    .withColumn(
        "Age_Category",
        F.when(F.col("Age_of_Driver").isin(1, 2), "Adolescente")
         .when(F.col("Age_of_Driver").isin(3, 4), "Joven")
         .when(F.col("Age_of_Driver").isin(5, 6), "Adulto")
         .when(F.col("Age_of_Driver").isin(7, 8), "Anciano")
    )

    # Extraigo la hora del accidente, desde la columna 'Datetime'.
    .withColumn("hora", F.hour("Datetime"))

    # Cacheo el DataFrame, dado que se reutilizará en ejercicios posteriores.
    .cache()
)

In [0]:
from pyspark.sql.types import DoubleType
assert(accidentesDF.schema[1].dataType == DoubleType())
assert(accidentesDF.count() == 251832)

assert(dict(accidentesDF.dtypes)["Age_Category"] == "string")
collectedDF = accidentesDF.groupBy("Age_Category").count().orderBy("count").collect()
assert((collectedDF[0]["count"] == 22533) & (collectedDF[0]["Age_Category"] == "Anciano"))
assert((collectedDF[1]["count"] == 57174) & (collectedDF[1]["Age_Category"] == "Adolescente"))
assert((collectedDF[2]["count"] == 67138) & (collectedDF[2]["Age_Category"] == "Adulto"))
assert((collectedDF[3]["count"] == 104987) & (collectedDF[3]["Age_Category"] == "Joven"))

**Ejercicio 2 (3 puntos)** 

Partiendo de `accidentesDF`, queremos pegar (**sin hacer JOIN sino usando agregaciones sobre ventanas**) a cada accidente la siguiente información:
* Número de accidentes que ha habido *en ese mismo año con esa misma categoría de vehículo*, en una nueva columna `total_vehiculo_anio`.
* Número de accidentes que ha habido *en ese mismo año con esa misma categoría de vehículo y con esa misma situación (Junction Location)*, en una nueva columna `total_vehiculo_causa_anio`.
* Porcentaje (en tanto por uno) que supone el segundo dato sobre el primero, en la columna `porc_vehiculo_causa_anio`. Esta columna la podrás calcular tras haber calculado las dos anteriores, sin necesidad de utilizar ninguna ventana.
* Edad promedio de los accidentados *en ese mismo año con esa misma categoría de vehículo*, en una nueva columna `media_edad_vehiculo_anio`.
* Edad promedio de los accidentados *en ese mismo año con esa misma categoría de vehículo y con esa misma situación (Junction_Location)*, en una nueva columna `media_edad_vehiculo_causa_anio`.
* Desviación típica (función `F.stddev`) del dato anterior *en ese mismo año con ese mismo tipo de vehículo y con esa misma situación (Junction_Location)*, en una nuea columna `stddev_edad_vehiculo_causa_anio`.
* Guardar el DF resultante en una nueva variable llamada `accidentes_info_agregadaDF`.

PISTA: crear en las variables `ventana_vehiculo_anio` y `ventana_vehiculo_causa_anio` dos ventanas diferentes que definan los dos grupos distintos que intervienen en los cálculos anteriores (una de ellos con dos criterios y la otra con tres).

**Revisa el diccionario de variables y su significado para saber qué columnas debes utilizar**.

In [0]:
# Importo la clase Window de PySpark, que permite definir particiones (grupos)
# sobre los cuales, aplicar funciones de ventana como count, avg o stddev sin agrupar el DataFrame.
from pyspark.sql.window import Window

# Creo dos ventanas, para aplicar agregaciones sin perder el detalle por fila:
# 1. Agrupación por año y categoría del vehículo.
ventana_vehiculo_anio = Window.partitionBy("Year", "Vehicle_Category")

# 2. Agrupación por año, categoría del vehículo y ubicación del accidente (Junction_Location).
ventana_vehiculo_causa_anio = Window.partitionBy("Year", "Vehicle_Category", "Junction_Location")

# Enriquezco el DataFrame original, con estadísticas agregadas usando las ventanas definidas.
accidentes_info_agregadaDF = (
    accidentesDF
    # Total de accidentes por año y categoría de vehículo
    .withColumn("total_vehiculo_anio", F.count("*").over(ventana_vehiculo_anio))

    # Total de accidentes por año, categoría de vehículo y ubicación del accidente.
    .withColumn("total_vehiculo_causa_anio", F.count("*").over(ventana_vehiculo_causa_anio))

    # Proporción (en tanto por uno) entre los accidentes con esa ubicación y el total del grupo.
    .withColumn("porc_vehiculo_causa_anio", F.col("total_vehiculo_causa_anio") / F.col("total_vehiculo_anio"))

    # Edad promedio del conductor por año y tipo de vehículo.
    .withColumn("media_edad_vehiculo_anio", F.avg("Age_of_Driver").over(ventana_vehiculo_anio))

    # Edad promedio del conductor por año, tipo de vehículo y ubicación del accidente.
    .withColumn("media_edad_vehiculo_causa_anio", F.avg("Age_of_Driver").over(ventana_vehiculo_causa_anio))

    # Desviación estándar de la edad del conductor, en ese mismo grupo detallado.
    .withColumn("stddev_edad_vehiculo_causa_anio", F.stddev("Age_of_Driver").over(ventana_vehiculo_causa_anio))
)

# COMENTARIO GENERAL DEL CÓDIGO:
# Este bloque enriquece el DataFrame original ('accidentesDF') con nuevas columnas estadísticas,
# calculadas mediante funciones de ventana. Estas funciones permiten obtener, para cada accidente:
# - El número total de accidentes por año y tipo de vehículo,
# - El número total de accidentes por año, tipo de vehículo y ubicación del accidente (Junction_Location),
# - La proporción entre ambos (porcentaje en tanto por uno),
# - La edad media y desviación estándar, del conductor dentro de esos grupos.
# De este modo, se añade contexto agregado a cada fila, sin necesidad de agrupar ni perder detalle.


In [0]:
from pyspark.sql import functions as F
r = accidentes_info_agregadaDF.select(F.mean("total_vehiculo_anio").alias("total_vehiculo_anio"),
                                  F.mean("total_vehiculo_causa_anio").alias("total_vehiculo_causa_anio"),
                                  F.mean("porc_vehiculo_causa_anio").alias("porc_vehiculo_causa_anio"),
                                  F.mean("media_edad_vehiculo_causa_anio").alias("media_edad_vehiculo_causa_anio"),
                                 ).first()
assert(round(r.total_vehiculo_anio, 2) == 33843.98)
assert(round(r.total_vehiculo_causa_anio, 2) == 8185.52)
assert(round(r.porc_vehiculo_causa_anio, 2) == 0.25)
assert(round(r.media_edad_vehiculo_causa_anio, 2) == 3.90)

**Ejercicio 3 (1 punto)** Queremos saber si el tipo de vehículo está relacionado con la hora del día a la que se tienen más accidentes, y si es diferente entre cada tipo de vehículo. Para ello, partiendo de nuevo de `accidentesDF` 
* Crear un nuevo DF con tantas filas como horas del día existen, y tantas columnas como categorías de vehículo más una (que será justamente la hora). En cada casilla, debe contener el número de accidentes ocurridos a esa hora del día con ese tipo de vehículo.
* Ordenar el DF en base a la hora de menor a mayor.
* Almacenar el DF resultante en la variable `accidentes_hora_vehiculo`.

In [0]:
# Construyo un nuevo DataFrame, que resume el número de accidentes por hora del día y tipo de vehículo.
# La columna 'hora' actuará como índice, y cada tipo de vehículo será una columna separada.

accidentes_hora_vehiculo = (
    accidentesDF

    # Agrupo los datos por la columna 'hora' (hora del día en que ocurrió el accidente).
    .groupBy("hora")

    # Aplico un pivot sobre la columna 'Vehicle_Category' para que cada tipo de vehículo se convierta en una columna.
    # Esto permite ver, en cada hora, cuántos accidentes hubo, de cada tipo de vehículo.
    .pivot("Vehicle_Category")
    # Cuento cuántos accidentes hay en cada combinación, hora-tipo de vehículo.
    .count()
    # Ordeno las filas por la hora de forma ascendente (de 0 a 23).
    .orderBy("hora")
)
# COMENTARIO GENERAL DEL CÓDIGO:
# Este bloque, analiza la distribución de accidentes según la hora del día y el tipo de vehículo.
# Crea una tabla con una fila por hora (0 a 23) y una columna por categoría de vehículo,
# donde cada celda contiene el número de accidentes ocurridos a esa hora, con ese tipo de vehículo.
# El resultado permite comparar visualmente en qué franjas horarias cada tipo de vehículo sufre más accidentes.


In [0]:
acc = accidentes_hora_vehiculo.collect()
assert(acc[0].hora == 0 and acc[0].Taxi == 324)
assert(acc[10].hora == 10 and acc[10].Other == 41)
assert(acc[15].hora == 15 and acc[15].Motorcycle == 1860)
assert(acc[19].hora == 19 and acc[19].Car == 10886)

**Ejercicio 4 (1 punto)** Partiendo de la variable `accidentes_hora_vehiculo` creada en el ejercicio anterior, crear un nuevo DF de **una sola fila** y tantas columnas como categorías de vehículos (es decir, 6). Debe contener, para cada columna, una *pareja del número de accidentes máximo que ocurre a lo largo del día, y la hora a la que se produjeron*. Para ello, en lugar de ir aplicando la función `F.max` a cada columna del DF anterior (dentro de una llamada a `select`), aplícala en cada momento lo que devuelve la función `F.struct(nombreColumna, "hora"`), es decir, `F.max(F.struct(nombreColumna, "hora"))`. De esta forma, estarás creando (al vuelo) un objeto columna de parejas, cuyo primer elemento de cada pareja es el número total de accidentes indicado en esa columna, y cuyo segundo elemento es la hora del día a la que se ha producido. La función `F.max` aplicada a una columna de tipo parejas tendrá en cuenta, por defecto, solamente el primer elemento de cada pareja para ordenar, así que escogerá la pareja que tiene un mayor número de accidentes ya que ese valor es el primer elemento de cada pareja, pero lo mostrará como pareja, con lo que veremos la hora del día a la que va aparejado ese número de accidentes.

Cada columna de pares mostrada por F.max debe renombrarse exactamente con el nombre de la categoría de vehículo a la que corresponde esa pareja. 

El DF resultante debe quedar guardado en la variable `hora_max_accidentes_vehiculo_df`

PISTA: la solución es simplemente una operación `select` que incluye dentro la creación de 6 columnas al vuelo haciendo 6 llamadas a la función `F.max(F.struct(..., "hora"))`, y haciendo `alias` sobre el objeto columna devuelto por cada una de estas llamadas, para que cada una de esas 6 columnas creadas al vuelo se llame igual que su categoría de vehículo.

In [0]:
# Creo un nuevo DataFrame, que tendrá una sola fila y 6 columnas (una por tipo de vehículo).
# En cada columna, guardo una estructura (struct) con dos elementos:
#  - El número máximo de accidentes registrados, para ese tipo de vehículo.
#  - La hora del día (0-23), en que ocurrió ese máximo.

hora_max_accidentes_vehiculo_df = accidentes_hora_vehiculo.select(

    # Para "Bus/minibus", obtenego la fila con más accidentes y guardo el par (accidentes, hora).
    F.max(F.struct("Bus/minibus", "hora")).alias("Bus/minibus"),

     # Para "Car" hago lo mismo: elijo el struct, con más accidentes y la hora correspondiente.
    F.max(F.struct("Car", "hora")).alias("Car"),

     # Para "Motorcycle" hago lo mismo.
    F.max(F.struct("Motorcycle", "hora")).alias("Motorcycle"),

    # Para "Other" hago lo mismo.
    F.max(F.struct("Other", "hora")).alias("Other"),

    # Para "Taxi" hago lo mismo.
    F.max(F.struct("Taxi", "hora")).alias("Taxi"),

    # Para "Van" hago lo mismo.
    F.max(F.struct("Van", "hora")).alias("Van")
)
# COMENTARIO GENERAL DEL CÓDIGO:
# Este bloque calcula, para cada categoría de vehículo, la hora del día en la que se produce
# el mayor número de accidentes. El resultado es un DataFrame de una sola fila, donde cada columna
# contiene un par (número de accidentes, hora) correspondiente al pico máximo de siniestros diarios.


In [0]:
assert(len(hora_max_accidentes_vehiculo_df.columns) == 6)
assert(sum([1 for c in ["Bus/minibus", "Car", "Motorcycle", "Other", "Taxi", "Van"]
          if c in hora_max_accidentes_vehiculo_df.columns]) == 6)
r2 = hora_max_accidentes_vehiculo_df.first()
assert(r2["Bus/minibus"][0] == 56 and r2["Bus/minibus"][1] == 15)
assert(r2["Car"][0] == 19961 and r2["Car"][1] == 17)
assert(r2["Motorcycle"][0] == 2751 and r2["Motorcycle"][1] == 17)
assert(r2["Other"][0] == 64 and r2["Other"][1] == 13)
assert(r2["Taxi"][0] == 389 and r2["Taxi"][1] == 16)
assert(r2["Van"][0] == 1233 and r2["Van"][1] == 16)

**Ejercicio 5 (2 puntos)** Vamos a preprocesar algunas variables para prepararlas para un posible algoritmo predictivo. Partiendo de `accidentesDF` se pide:
* Crear en la variable `journey_purpose_indexer` un StringIndexer para la variable "Driver_Journey_Purpose" y que cree una nueva columna de salida `purpose_indexed`. Debe ser capaz de lidiar con etiquetas nunca vistas a la hora de hacer la codificación de un nuevo dataset (que no se eliminen dichas filas ni tampoco salte un error).
* Crear en la variable `cars_involved_binarizer` un Binarizer de la variable `Number_of_Vehicles` que tenga `threshold=2.5` puesto que en la mayoría de los accidentes están involucrados 1 o 2 coches. Queremos pasarla a una variable binaria donde el 0.0 represente justamente que ha habido 1 o 2 coches involucrados, y el 1.0 represente que ha habido 3 o más coches involucrados. El binarizador debe crear como salida una nueva columna llamada `number_vehicles_binarized`
* Crear en la variable `vector_assembler` un VectorAssembler que colapse en una nueva columna de tipo vector las columnas `purpose_indexed`, `number_vehicles_binarized` y `Speed_limit`. La nueva columna debe llamarse `features`. 
* Crear en la variable `pipeline` un pipeline que contenga **exclusivamente** las tres etapas anteriores. **NO DEBE CONTENER NINGÚN ALGORITMO PREDICTIVO**.
* "Entrenar" ese pipeline con el DF `accidentesDF` y guardar el resultado en la variable `pipeline_model`. **No debe hacerse ningún tipo de división de los datos en entrenamiento y test**. Aunque el método sea "entrenar", en realidad sólo estamos ajustando etapas de pre-procesamiento.

In [0]:
# Importo las clases específicas, de la librería pyspark.ml.feature que permiten transformar columnas,
# en formatos que los modelos de Machine Learning pueden procesar.
from pyspark.ml.feature import StringIndexer, Binarizer, VectorAssembler
from pyspark.ml import Pipeline

# Creo un StringIndexer, para transformar la columna "Driver_Journey_Purpose" (texto),
# en una columna numérica llamada "purpose_indexed".
# Uso handleInvalid="keep" para evitar errores, si aparece una categoría no vista al entrenar.
journey_purpose_indexer = StringIndexer(
    inputCol="Driver_Journey_Purpose", # Columna de entrada (string).
    outputCol="purpose_indexed",       # Columna de salida (númerica).
    handleInvalid="keep"               # Si hay valores nuevos en datos futuros, no da error.
)

# Creo un Binarizer, para convertir el número de vehículos en una variable binaria.
# Si el número de vehículos es mayor a 2.5, se pone 1.0, si no, se pone 0.0.
cars_involved_binarizer = Binarizer(
    inputCol="Number_of_Vehicles",           # Entrada: número de vehículos.
    outputCol="number_vehicles_binarized",   # Salida: 0.0 o 1.0.
    threshold=2.5                            # Umbral: 3 o más vehículos → 1.0.
)

# Creo un VectorAssembler, el cual combina 3 columnas en una sola columna de vectores numéricos.
# Esta columna se llama "features" y se usará para alimentar algoritmos de ML.
vector_assembler = VectorAssembler(
    inputCols=["purpose_indexed", "number_vehicles_binarized", "Speed_limit"],  # Entradas.
    outputCol="features"                                                        # Salida.
)

# Hago la unión de los tres pasos anteriores, en un único pipeline.
# El pipeline ejecutará esas transformaciones en secuencia.
pipeline = Pipeline(stages=[
    journey_purpose_indexer,     # 1. Converte texto a número.
    cars_involved_binarizer,     # 2. Converte número de vehículos a binario.
    vector_assembler             # 3. Junta las columnas en un solo vector.
])

# "Entreno" el pipeline con el DataFrame original.
# No entrena un modelo, solo ajusta los transformadores, para que estén listos para aplicar.
pipeline_model = pipeline.fit(accidentesDF)

# COMENTARIO GENERAL DEL CÓDIGO:
# Este bloque construye un pipeline de preprocesamiento sobre el DataFrame 'accidentesDF',
# preparando las variables para su uso en modelos predictivos. Convierte una variable categórica
# en índice numérico, transforma una variable continua en binaria, y combina todas en un vector
# de entrada. Estas etapas se encapsulan en un Pipeline, que puede aplicarse fácilmente a nuevos datos.


In [0]:
from pyspark.ml.feature import StringIndexer, Binarizer, VectorAssembler
from pyspark.ml import Pipeline, PipelineModel

assert(isinstance(journey_purpose_indexer, StringIndexer))
assert(journey_purpose_indexer.getInputCol() == "Driver_Journey_Purpose" and 
       journey_purpose_indexer.getOutputCol() == "purpose_indexed" and
       journey_purpose_indexer.getHandleInvalid() == "keep")

assert(isinstance(cars_involved_binarizer, Binarizer))
assert(cars_involved_binarizer.getInputCol() == "Number_of_Vehicles" and 
       cars_involved_binarizer.getOutputCol() == "number_vehicles_binarized" and
       cars_involved_binarizer.getThreshold() == 2.5)

assert(isinstance(pipeline, Pipeline))
assert(len(pipeline.getStages()) == 3)             # el pipeline debe tener solamente tres etapas
assert(journey_purpose_indexer in pipeline.getStages() 
       and cars_involved_binarizer in pipeline.getStages() 
       and vector_assembler in pipeline.getStages())

assert(isinstance(pipeline_model, PipelineModel))

**Ejercicio 6 (1.5 puntos)** Queremos ver cuál es la forma de transporte involucrada en más accidentes en cada región de Reino Unido. Para ello, partiendo de `accidentesDF` se pide:

* Crear un DF con tantas filas como Regiones distintas existen y tantas columnas como categorías de vehículo más una (la de la región, que estará a la izquierda). En cada casilla debe calcularse una **tripleta** (columna de tipo estructura, que se crea con `F.struct(col1, col2, col3)`) formada por:
  * Número de accidentes en esa región y tipo vehículo,
  * Edad media del conductor redondeada a 2 cifras decimales, y
  * Número medio de coches involucrados redondeado a 2 cifras decimales). 
* Ordenar el DF alfabéticamente de menor a mayor en base a la columna `"Region"`
* Guardar el DF resultante en la variable `numero_edad_coches_df`.
* Para visualizarlo mejor, y puesto que el tamaño del DF que hemos obtenido como resultado está acotado por el número de regiones distintas existentes y por el número de categorías de vehículos existentes, pasar dicho DF a un dataframe de Pandas en la variable `numero_edad_coches_pd` y mostrarlo por pantalla.

PISTA: para construir la columna de tipo estructura dentro de la función `agg(...)` se puede utilizar `F.struct(F.funcionagregacion(...), F.funcionagregacion(...), F.funcionagregacion(...))`. 

PISTA: en vez de pasarle a la función `F.struct` directamente la columna resultante de la agregación, pásale en caso necesario `F.round(F.nombrefuncion(...), 2)` para que ya esté redondeada.

En el resultado puedes observar fenómenos como por ejemplo: 
* Los conductores de autobús son los que en promedio tienen siempre más edad, mientras que los de moto son los más jóvenes, como era de esperar. 
* Los accidentes de moto son los que menos coches involucran en promedio, en torno a 1.80, lo que quiere decir que hay muchos accidentes que los tiene el propio conductor sin que intervenga otro vehículo (condiciones atmosféricas, etc). Los de Taxi parecen estar bastante por debajo que los accidentes de coche, lo que indica que, mientras que un accidente de coche con frecuencia implica la interacción con otro vehículo, en los taxis hay aún bastantes accidentes donde no necesariamente hay otro vehículo implicado y por eso el promedio todavía no se acerca tanto a 2.
* Es llamativo que en Gales haya unos promedios tan bajos en el número de vehículos involucrados en todas las categorías, en especial en los accidentes de coche y moto, lo que indica que muchos accidentes se producen sin causa de otro vehículo. Puede tener relación directa con que la edad media de los conductores es bastante superior a la de otros medios, y por eso son propensos a tener un accidente por mala conducción o relacionado con las facultades físicas o cognitivas del conductor.

In [0]:
# Agrupo el DataFrame original, por Región y Tipo de Vehículo
# y calculo tres métricas para cada combinación:
# 1. Número total de accidentes.
# 2. Edad promedio del conductor (redondeada a 2 decimales).
# 3. Promedio de vehículos implicados (redondeado a 2 decimales).
# Luego empaqueto estas 3 métricas, en una sola estructura (struct).
grupo = accidentesDF.groupBy("Region", "Vehicle_Category").agg(
    F.struct(
        F.count("*"),                               # Total de accidentes.
        F.round(F.avg("Age_of_Driver"), 2),         # Edad media del conductor.
        F.round(F.avg("Number_of_Vehicles"), 2)     # Promedio de vehículos implicados.
    ).alias("resumen")                              # Le doy el nombre "resumen" a esta estructura.
)

# A partir del resultado anterior, hago la agrupación solo por Región
# y aplico pivot, para convertir los tipos de vehículos en columnas individuales.
# Para cada combinación, selecciono el primer valor del resumen (ya que solo hay uno por grupo).
numero_edad_coches_df = (
    grupo
    .groupBy("Region")           # Agrupo solo por región
    .pivot("Vehicle_Category")   # Cada tipo de vehículo, será una columna.
    .agg(F.first("resumen"))     # Tomo la estructura calculada, para cada tipo
    .orderBy("Region")           # Ordeno alfabéticamente, por Región.
)

# Convierto el DataFrame de Spark, a un DataFrame de Pandas
# para poder visualizarlo más fácilmente, como tabla o exportarlo si fuera necesario.
numero_edad_coches_pd = numero_edad_coches_df.toPandas()

# Mostro por pantalla el resultado final en formato Pandas.
# Cada celda contiene una tupla: (nº de accidentes, edad media, nº vehículos)
print(numero_edad_coches_pd)

# COMENTARIO GENERAL DEL CÓDIGO:
# Este bloque analiza los accidentes por región y tipo de vehículo.
# Para cada combinación se calcula el número de accidentes, la edad media del conductor
# y el promedio de vehículos implicados. Estos datos se empaquetan en estructuras (structs)
# y se pivotan para formar un resumen por región, con columnas separadas por tipo de vehículo.
# Finalmente, se convierte a Pandas para facilitar su visualización.


                      Region                               Bus/minibus  \
0               East England  {'col1': 44, 'col2': 4.34, 'col3': 1.77}   
1              East Midlands  {'col1': 39, 'col2': 4.95, 'col3': 1.85}   
2                     London  {'col1': 28, 'col2': 4.71, 'col3': 1.75}   
3         North East England  {'col1': 52, 'col2': 4.46, 'col3': 1.85}   
4         North West England   {'col1': 66, 'col2': 4.7, 'col3': 1.88}   
5                   Scotland     {'col1': 2, 'col2': 4.0, 'col3': 1.5}   
6         South East England  {'col1': 71, 'col2': 4.96, 'col3': 1.87}   
7         South West England  {'col1': 37, 'col2': 4.84, 'col3': 1.81}   
8                      Wales     {'col1': 1, 'col2': 5.0, 'col3': 2.0}   
9              Wast Midlands   {'col1': 56, 'col2': 4.7, 'col3': 1.91}   
10  Yorkshire and the Humber  {'col1': 75, 'col2': 4.67, 'col3': 1.69}   

                                            Car  \
0    {'col1': 22813, 'col2': 3.95, 'col3': 2.0}   
1   {'col

In [0]:
assert(len(numero_edad_coches_df.columns) == 7)
assert(sum([1 for c in ["Region", "Bus/minibus", "Car", "Motorcycle", "Other", "Taxi", "Van"]
          if c in numero_edad_coches_df.columns]) == 7)
r = numero_edad_coches_df.collect()
assert(r[0].Region == "East England" and r[0].Other == (61, 4.02, 1.87))
assert(len(numero_edad_coches_df.columns) == 7 and len(r) == 11)
assert(r[0].Car == (22813, 3.95, 2.0))
assert(r[7].Motorcycle == (2802, 3.17, 1.88))
assert(r[10].Van == (1412, 3.92, 2.02))